In [178]:
import numpy as np
import pandas as pd

from tqdm import tqdm
import os
import pickle as pkl
import time
import nltk
from textblob import TextBlob
import nltk
nltk.download('vader_lexicon')


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/ericsaikali/nltk_data...


True

In [160]:
DATA_FOLDER = 'data/MovieSummaries/'
ADDITIONAL_FOLDER = 'data/AdditionalData/'

In [161]:
# reading a txt file and convert it to a dataframe
def read_txt(path):
    df = pd.read_csv(path, sep='\t', header=None)
    return df

In [162]:
# importing the data
plots = pd.read_csv(DATA_FOLDER + 'plot_summaries.txt', header=None, sep="\t")
movies = pd.read_csv(DATA_FOLDER + 'movie.metadata.tsv', header=None, sep="\t")
characters = pd.read_csv(DATA_FOLDER + 'character.metadata.tsv', header=None, sep="\t")
names = pd.read_csv(DATA_FOLDER + 'name.clusters.txt', header=None, sep="\t")
tvtropes = pd.read_csv(DATA_FOLDER + 'tvtropes.clusters.txt', header=None, sep="\t")

In [163]:
# renaming columns the plots dataframe
plots.columns = ['wikipedia_movie_id', 'plot']

In [175]:
# Sample synopsis
synopsis = plots.iloc[:, 1].loc[0]

In [176]:
def find_ending(plot, max_sentences=3):
    blob = TextBlob(plot)
    last_sentences = blob.sentences[-max_sentences:]  # Adjust the number of sentences as needed
    return " ".join([str(sentence) for sentence in last_sentences])

In [177]:
plots['endings'] = plots.iloc[:, 1].apply(lambda plot: find_ending(plot))
plots['endings']

0        Shlykov, a hard-working taxi driver and Lyosha...
1        However, before they can commit suicide, they ...
2        At Menon's funeral, Manapally Pavithran arrive...
3        Moran and Charley are arrested while the judge...
4        In October 1982, Lindy is found guilty and sen...
                               ...                        
42298    The story is about Reema , a young Muslim scho...
42299    Moved, Leo tells him they have. The assistant ...
42300    Now Parsons is creating instruments that are h...
42301    Mini does not recognize Rehman, who realises t...
42302    Mainwaring and his men become the pride of the...
Name: endings, Length: 42303, dtype: object

In [184]:
from nltk.sentiment import SentimentIntensityAnalyzer

sia = SentimentIntensityAnalyzer()
text = "I love Python. It's such an amazing language!"

sentiment = sia.polarity_scores(text)
print(sentiment)


{'neg': 0.0, 'neu': 0.376, 'pos': 0.624, 'compound': 0.8516}


In [181]:
plots['ending_sentiment'] = plots['endings'].apply(lambda end: sia.polarity_scores(end)['compound'])


In [182]:
plots['ending_sentiment'] 

0        0.0083
1       -0.7269
2       -0.5719
3        0.7096
4       -0.9042
          ...  
42298    0.2732
42299    0.9231
42300    0.9686
42301    0.4215
42302   -0.1779
Name: ending_sentiment, Length: 42303, dtype: float64

count    42303.000000
mean        -0.043300
std          0.655356
min         -0.997300
25%         -0.695600
50%         -0.025800
75%          0.599400
max          0.997000
Name: ending_sentiment, dtype: float64